In [2]:
%cd /content/drive/MyDrive/job/recommender_models/components/lightgbm_ranker

/content/drive/MyDrive/job/recommender_models/components/lightgbm_ranker


In [81]:
import pandas as pd
import numpy as np
from datetime import datetime
import lightgbm as lgb

In [76]:
train_data_path = '../../data/train.pkl'
valid_data_path = '../../data/valid.pkl'
features = ['price']
target = 'buy'
indexes = ['t_dat', 'customer_id']

In [77]:
train = pd.read_pickle(train_data_path)
valid = pd.read_pickle(valid_data_path)

In [106]:
sample_weight=[]
train_group=[]
valid_group=[]

In [107]:
params = {
    'objective': "lambdarank",
    'boosting_type': "dart",
    'metric': "ndcg",
    'num_leaves': 2**7-1,
    'max_depth': 7,
    'num_boost_round': 100,
    'early_stopping_rounds': 10,
    'verbosity': 10,
    'random_state': 42,
    'importance_type': 'gain',
}

In [ ]:
model = lgb.LGBMRanker(params)

In [ ]:
model.fit(
    X=train[features],
    y=train[target],
    sample_weight=sample_weight,
    group=train_group,
    eval_set=(valid[features], valid[target]),
    eval_names=['valid'],
    eval_at=[1],
    eval_group=valid_group,
    )